# Finetuning Huggingface models on Azure ML

This notebook explains how to create an end-to-end lineage for hugging face models on Azure.

In [1]:
from azure.identity import DefaultAzureCredential, AzureCliCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import AmlCompute
from azure.ai.ml.entities import Environment, BuildContext
from azure.ai.ml import command, Input, Output
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import (
    VsCodeJobService,
    TensorBoardJobService,
    JupyterLabJobService,
)

### Create AML Compute

In [2]:
# Function to create MLClient
def create_ml_client(credential=None):     
    if credential is None:
        try:
            credential = DefaultAzureCredential()
            # Check if given credential can get token successfully.
            credential.get_token("https://management.azure.com/.default")
        except Exception as ex:
            # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
            credential = AzureCliCredential()

    return MLClient.from_config(credential=credential)

# Function to create AML Compute Cluster
def create_aml_cluster(ml_client, compute_cluster_name = "AmlComputeCluster", vm_size = "Standard_NC4as_T4_v3", min_nodes = 1, max_nodes = 2):
    # If you already have a gpu cluster, mention it here. Else will create a new one    
    try:
        compute = ml_client.compute.get(compute_cluster_name)
        print("successfully fetched compute:", compute.name)
    except Exception as ex:
        print("failed to fetch compute:", compute_cluster_name)
        print(f"creating new {vm_size} compute")
        compute = AmlCompute(
            name=compute_cluster_name,
            size=vm_size,
            min_instances=min_nodes,
            max_instances=max_nodes,  # For multi node training set this to an integer value more than 1
        )
        ml_client.compute.begin_create_or_update(compute).wait()
        print("successfully created compute:", compute.name)
    return compute

### Create Custom Environment

In [10]:
# Function to create Environment
def create_environment(ml_client, env_Name = "finetune_hf_lora"):
    try:
        env = ml_client.environments.get(env_Name)
        print("successfully fetched environment:", env.name)
    except Exception as ex:
        print("failed to fetch environment:", env_Name)
        print(f"creating new environment {env_Name}")
        env_docker_context = Environment(
            build=BuildContext(path="src/env"),
            name=env_Name,
            description="Environment created from a Docker context.",
        )
        ml_client.environments.create_or_update(env_docker_context)
        print("successfully created environment:", env_docker_context.name)
    return env_docker_context

### Create Azure ML Dataset

In [4]:
# Create Azure ML dataset from local file
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

VERSION = "1.0"
NAME = "squad_dev_v1"
def create_dataset(ml_client):
    # Create a dataset from local file
    dataset = Data(
        type=AssetTypes.URI_FILE,
        path="./data/squad.json",
        description="SquAD v1.0 dev dataset",
        name=NAME,
        version=VERSION,
    )
    return ml_client.data.create_or_update(dataset)

ml_client = create_ml_client()
try:
   ml_client.data.get(NAME, VERSION)
except:
    create_dataset(ml_client)

Found the config file in: ./config.json


### Create Azure ML Job

In [5]:
# from azure.ai.ml.constants import AssetTypes, InputOutputModes
# # Set the input for the job:
# data_asset = ml_client.data.get(NAME, version=VERSION)
# inputs = Input(path=data_asset.id, mode=InputOutputModes.MOUNT, type=AssetTypes.URI_FILE, destination_path_on_compute="data")
# mlflow_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri

# # Function to create Job
# def create_job(compute_cluster, script_file = "finetune_hf_models.py", job_name = "hf_finetuning", env_name = "finetune_hf_lora"):
#     job = command(
#         code=".",
#         command=f"python src/{script_file} \
#             --model_name google/flan-t5-small \
#             --num_epochs 1 \
#             --mlflow_tracking_uri {mlflow_tracking_uri} \
#             --data_path ${{inputs.data}} \
#             --target_input_length=512 \
#             --target_max_length=100 \
#             --train_size=1000",
#     compute=compute_cluster,
#     inputs={
#       "data": inputs
#     },
#     services={
#       "My_jupyterlab": JupyterLabJobService(
#         nodes="all" # For distributed jobs, use the `nodes` property to pick which node you want to enable interactive services on. If `nodes` are not selected, by default, interactive applications are only enabled on the head node. Values are "all", or compute node index (for ex. "0", "1" etc.)
#       ),
#       "My_vscode": VsCodeJobService(
#         nodes="all"
#       ),
#       "My_tensorboard": TensorBoardJobService(
#         nodes="all",
#         log_dir="outputs/runs"  # relative path of Tensorboard logs (same as in your training script)         
#       ),
#     },
#     environment=f"{env_name}@latest",
#     instance_count=1,  
#     display_name=job_name)
#     return job

NameError: name 'ml_client' is not defined

In [58]:
# compute_cluster_name = "AmlComputeCluster"
# ml_client = create_ml_client()
# compute_cluster = create_aml_cluster(ml_client, compute_cluster_name = compute_cluster_name)
# env_docker_context = create_environment(ml_client)
# job = create_job(compute_cluster_name)
# print(f"Creating job {job.display_name}")
# # Submit the job and wait for completion
# job = ml_client.jobs.create_or_update(job, show_output = True)
# print("Job created successfully")
# print(job.studio_url)   

Found the config file in: ./config.json


successfully fetched compute: AMLComputeCluster
failed to fetch environment: finetune_hf_lora
creating new environment finetune_hf_lora
successfully created environment: finetune_hf_lora
Creating job hf_finetuning


Uploading finetuning_hf_models (85.26 MBs): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 85262513/85262513 [00:00<00:00, 115392488.33it/s]


Job created successfully
https://ml.azure.com/runs/willing_avocado_h5g2k7n8gc?wsid=/subscriptions/6a01260f-39d6-415f-a6c9-cf4fd479cbec/resourcegroups/sriks-ml-rg/workspaces/sriks-ml-sea&tid=16b3c013-d300-468d-ac64-7eda0820b6d3


### Create Azure ML pipeline job

In [33]:
from src.prep.prepare_component import prepare_data_component
from src.train.train_component import training_component

help(prepare_data_component)
help(training_component)

Help on function prepare_data_component in module src.prep.prepare_component:

prepare_data_component()

Help on function training_component in module src.train.train_component:

training_component(input_data: <mldesigner._input_output.Input object at 0x7f3dbb7907f0>, output_model: <mldesigner._input_output.Output object at 0x7f3dbb790d30>)



In [34]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.constants import AssetTypes, InputOutputModes

compute_cluster_name = "AmlComputeCluster"
env_name = "finetune_hf_lora"
data_asset = ml_client.data.get(NAME, version=VERSION)

ml_client = create_ml_client()
mlflow_tracking_uri = ml_client.workspaces.get(ml_client.workspace_name).mlflow_tracking_uri
compute_cluster = create_aml_cluster(ml_client, compute_cluster_name = compute_cluster_name)
env_docker_context = create_environment(ml_client)

Found the config file in: ./config.json


successfully fetched compute: AMLComputeCluster
failed to fetch environment: finetune_hf_lora
creating new environment finetune_hf_lora
successfully created environment: finetune_hf_lora


In [37]:
# Define the data preparation component using python SDK
# Alternatively we can declare components using YAML
# The component is defined using the command function
data_prep_component = command(
    name="data_prep_qna_squad_dev_v1",
    display_name="Data preparation for fine tuning HF models on SQuAD v1.0 dev dataset",
    description="Reads the SQuAD v1.0 dev dataset and prepares it for fine tuning HF models",
    inputs={
        "data": Input(mode=InputOutputModes.MOUNT, type=AssetTypes.URI_FILE, destination_path_on_compute="data")
    },
    outputs=dict(
        train_data=Output(type="uri_folder", mode="rw_mount")
    ),
    # The source folder of the component
    code='src/prep',
    command="""python prepare_component.py \
            --data_path ${{inputs.data}} \
            --target_input_length=512 \
            --target_max_length=100 \
            --train_size=1000""",
    environment=f"{env_name}@latest",
)

# Now we register the component to the workspace
data_prep_component = ml_client.create_or_update(data_prep_component.component)
print(f"Component {data_prep_component.name} with Version {data_prep_component.version} is registered")

ValidationException: {
  "result": "Failed",
  "errors": [
    {
      "message": "Missing data for required field.",
      "path": "environment",
      "value": null
    }
  ]
}

In [36]:
inputs = Input(path=data_asset.id, mode=InputOutputModes.MOUNT, type=AssetTypes.URI_FILE, destination_path_on_compute="data")

@pipeline(
    default_compute=compute_cluster_name,
)
def finetune_hfmodels_azureml_pipeline(pipeline_input_data):
    """E2E Hugging face Q and A model using huggingface, peft, azureml and python sdk."""
    prepare_data_node = data_prep_component(data=pipeline_input_data)

# create a pipeline
pipeline_job = finetune_hfmodels_azureml_pipeline(pipeline_input_data=inputs)
pipeline_job = ml_client.jobs.create_or_update(pipeline_job, experiment_name="finetuning_hf")
ml_client.jobs.stream(pipeline_job.name)


RunId: joyful_hat_yy7f6gcvqw
Web View: https://ml.azure.com/runs/joyful_hat_yy7f6gcvqw?wsid=/subscriptions/6a01260f-39d6-415f-a6c9-cf4fd479cbec/resourcegroups/sriks-ml-rg/workspaces/sriks-ml-sea

Streaming logs/azureml/executionlogs.txt

[2024-04-09 17:17:02Z] Submitting 1 runs, first five are: 226a0f41:ea7ce886-d1d6-4097-b6f3-71f2dddc583e
[2024-04-09 17:17:44Z] Execution of experiment failed, update experiment status and cancel running nodes.

Execution Summary
RunId: joyful_hat_yy7f6gcvqw
Web View: https://ml.azure.com/runs/joyful_hat_yy7f6gcvqw?wsid=/subscriptions/6a01260f-39d6-415f-a6c9-cf4fd479cbec/resourcegroups/sriks-ml-rg/workspaces/sriks-ml-sea


JobException: Exception : 
 {
    "error": {
        "code": "UserError",
        "message": "Pipeline has failed child jobs. Failed nodes: /prepare_data_node. For more details and logs, please go to the job detail page and check the child jobs.",
        "message_format": "Pipeline has failed child jobs. {0}",
        "message_parameters": {},
        "reference_code": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "southeastasia",
    "location": "southeastasia",
    "time": "2024-04-09T17:17:44.604192Z",
    "component_name": ""
} 